![piggy_bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Read csv file 

In [2]:
df = pd.read_csv(r'C:\Users\ojoar\Desktop\data_set\bank_marketing\bank_marketing.csv')

### Standardized data from the column

In [10]:
# Clean education column
df["education"] = np.where(
    df["education"].str.replace('.', ' ', regex=False).str.title().str.startswith("Basic"),
    df["education"].str.replace('.', ' ', regex=False).str.title().str.split().str[0],
    df["education"].str.replace('.', ' ', regex=False).str.title()
)                        

In [11]:
df["education"].unique()

array(['Basic', 'High School', 'Professional Course', 'Unknown',
       'University Degree', 'Illiterate'], dtype=object)

In [19]:
# Clean job column
df["job"] = df["job"].str.replace("_", " ")\
                     .str.capitalize()\
                     .str.strip()

# Clean and convert the credit_default and mortgage columns to bool data type
for col in ["credit_default", "mortgage"]:
  df[col] = df[col].map({"yes": 1,
                                 "no": 0,
                                 "unknown": 0})
  df[col] = df[col].astype(bool)

In [22]:
# Change campaign_outcome to binary values
df["campaign_outcome"] = df["campaign_outcome"].map({"yes": 1, 
                                                                 "no": 0})

# Convert previous_outcome to binary values
df["previous_outcome"] = df["previous_outcome"].map({"success": 1, 
                                                                 "failure": 0,
                                                                 "nonexistent": 0})

# Add year column
df["year"] = "2022"

# Convert day to string
df["day"] = df["day"].astype(str)

# Add last_contact_date column
df["last_contact_date"] =df["year"] + "-" + df["month"] + "-" + df["day"]

# Convert to datetime
df["last_contact_date"] = pd.to_datetime(df["last_contact_date"], 
                                               format="%Y-%b-%d")

# Clean and convert outcome columns to bool
for col in ["campaign_outcome", "previous_outcome"]:
  df[col] = df[col].astype(bool)

# Drop unneccessary columns
df.drop(columns=["month", "day", "year"], inplace=True)

In [23]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome,last_contact_date
0,0,56,Housemaid,married,Basic,True,True,261,1,0,False,93.994,4.857,False,2022-05-13
1,1,57,Services,married,High School,True,True,149,1,0,False,93.994,4.857,False,2022-05-19
2,2,37,Services,married,High School,True,True,226,1,0,False,93.994,4.857,False,2022-05-23
3,3,40,Admin,married,Basic,True,True,151,1,0,False,93.994,4.857,False,2022-05-27
4,4,56,Services,married,High School,True,True,307,1,0,False,93.994,4.857,False,2022-05-03


### Split into the three tables

In [24]:
client = df[["client_id", "age", "job", "marital", 
                    "education", "credit_default", "mortgage"]]
campaign = df[["client_id", "number_contacts","contact_duration",
                    "previous_campaign_contacts", "previous_outcome", "campaign_outcome"]]
economics = df[["client_id", "cons_price_idx", "euribor_three_months", "last_contact_date"]]

In [30]:
client.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,Housemaid,married,Basic,True,True
1,1,57,Services,married,High School,True,True
2,2,37,Services,married,High School,True,True
3,3,40,Admin,married,Basic,True,True
4,4,56,Services,married,High School,True,True


In [31]:
campaign.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,False,False
1,1,1,149,0,False,False
2,2,1,226,0,False,False
3,3,1,151,0,False,False
4,4,1,307,0,False,False


In [32]:
economics.head()

,client_id,cons_price_idx,euribor_three_months,last_contact_date
0,0,93.994,4.857,2022-05-13
1,1,93.994,4.857,2022-05-19
2,2,93.994,4.857,2022-05-23
3,3,93.994,4.857,2022-05-27
4,4,93.994,4.857,2022-05-03


### Save tables to individual csv files

In [33]:
client.to_csv("client.csv", index=False)
campaign.to_csv("campaign.csv", index=False)
economics.to_csv("economics.csv", index=False)